In [1]:
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd

df = pd.read_csv('lam_data_processed.csv')
df.head()

,Aika,N
0,2020-01-01 00:00:00,12650.0
1,2020-01-01 01:00:00,11666.0
2,2020-01-01 02:00:00,8620.0
3,2020-01-01 03:00:00,6654.0
4,2020-01-01 04:00:00,5784.0
